# Day 18

After the million lights incident, the fire code has gotten stricter: now, at most ten thousand lights are allowed. You arrange them in a 100x100 grid.

Never one to let you down, Santa again mails you instructions on the ideal lighting configuration. With so few lights, he says, you'll have to resort to animation.

Start by setting your lights to the included initial configuration (your puzzle input). A `#` means "on", and a `.` means "off".

Then, animate your grid in steps, where each step decides the next configuration based on the current one. Each light's next state (either on or off) depends on its current state and the current states of the eight lights adjacent to it (including diagonals). Lights on the edge of the grid might have fewer than eight neighbors; the missing ones always count as "off".

For example, in a simplified 6x6 grid, the light marked A has the neighbors numbered 1 through 8, and the light marked B, which is on an edge, only has the neighbors marked 1 through 5:

```text
1B5...
234...
......
..123.
..8A4.
..765.
```

The state a light should have next is based on its current state (on or off) plus the number of neighbors that are on:

    A light which is on stays on when 2 or 3 neighbors are on, and turns off otherwise.
    A light which is off turns on if exactly 3 neighbors are on, and stays off otherwise.

All of the lights update simultaneously; they all consider the same current state before moving to the next.

Here's a few steps from an example configuration of another 6x6 grid:

```text
Initial state:
.#.#.#
...##.
#....#
..#...
#.#..#
####..

After 1 step:
..##..
..##.#
...##.
......
#.....
#.##..

After 2 steps:
..###.
......
..###.
......
.#....
.#....

After 3 steps:
...#..
......
...#..
..##..
......
......

After 4 steps:
......
......
..##..
..##..
......
......
```

After 4 steps, this example has four lights on.

In your grid of 100x100 lights, given your initial configuration, how many lights are on after 100 steps?

## Puzzle 1

In [1]:
import numpy as np


class Lights():
    
    def __init__(self, initial_state, broken=False):
        self._broken = True
        self.__parse_state(initial_state)
                
    def update(self):
        """Update lights according to rules"""
        newarray = self._array.copy()
        for row in range(self._arraysize):
            for col in range(self._arraysize):
                nbrscore = (sum(sum(self._array[max(0, row-1):min(self._arraysize, row+2),
                                                max(0, col-1):min(self._arraysize, col+2)])) -
                            self._array[row, col])
                
                if nbrscore == 3 and self._array[row, col] == 0:
                    newarray[row, col] = 1
                elif nbrscore not in (2, 3) and self._array[row, col] == 1:
                    newarray[row, col] = 0

        if self._broken:
            for row, col in ((0, 0), (0, self._arraysize - 1),
                             (self._arraysize - 1, 0),
                             (self._arraysize - 1, self._arraysize - 1)):
                newarray[row, col] = 1
                    
        self._array = newarray
            
    def __parse_state(self, state:str):
        """Set up light state
        
        :param state: string representation of lights
        """
        # Initialise
        lines = state.split()

        rows = []
        for idx, line in enumerate(lines):
            row = [int(_) for _ in line.replace(".", "0").replace("#", "1")]
            rows.append(row)
        
        self._arraysize = len(rows)
        self._array = np.array(rows)

        # Corner lights fixed on
        if self._broken:
            for row, col in ((0, 0), (0, self._arraysize - 1),
                             (self._arraysize - 1, 0),
                             (self._arraysize - 1, self._arraysize - 1)):
                self._array[row, col] = 1
        
    def __str__(self):
        """Return string showing array"""
        outstr = []
        for row in self._array:
            outstr.append(str(row).replace("0", ".").replace("1", "#").replace(" ","")[1:-1])
        return "\n".join(outstr) + "\n"
    
    @property
    def brightness(self):
        """Return total number of lights that are on"""
        return sum(sum(self._array))

In [2]:
initial = """.#.#.#
...##.
#....#
..#...
#.#..#
####.."""

lights = Lights(initial)
print(lights)
for idx in range(4):
    lights.update()
    print(lights)

##.#.#
...##.
#....#
..#...
#.#..#
####.#

#.##.#
####.#
...##.
......
#...#.
#.####

#..#.#
#....#
.#.##.
...##.
.#..##
##.###

#...##
####.#
..##.#
......
##....
####.#

#.####
#....#
...#..
.##...
#.....
#.#..#



In [3]:
str(lights._array[0])

'[1 0 1 1 1 1]'

### Solution

In [4]:
with open("day18.txt", "r") as ifh:
    lights = Lights(ifh.read())

for idx in range(100):
    lights.update()

print(lights.brightness)
print(lights)

1006
##............................#......
.......................#.............
........................##
#.#..........................###.....
............#.#.........#............
........................##
.#..........................##.##....
............#.#.......#..#...........
..........................
...........................###.###...
...........#...#..###...#............
..........................
............................##.##....
............#..#..#.#.##.....#...##..
..........................
.............................###.....
............#..#.......#.....##..##..
...............###........
..............................#......
..............#.....###..........#...
..........................
.....................................
......................#..........###.
.............#......#.....
...................................##
.........##......................#..#
.............#.....#.#....
...................................##
........#.##..................

## Puzzle 2

You flip the instructions over; Santa goes on to point out that this is all just an implementation of Conway's Game of Life. At least, it was, until you notice that something's wrong with the grid of lights you bought: four lights, one in each corner, are stuck on and can't be turned off. The example above will actually run like this:

```text
Initial state:
##.#.#
...##.
#....#
..#...
#.#..#
####.#

After 1 step:
#.##.#
####.#
...##.
......
#...#.
#.####

After 2 steps:
#..#.#
#....#
.#.##.
...##.
.#..##
##.###

After 3 steps:
#...##
####.#
..##.#
......
##....
####.#

After 4 steps:
#.####
#....#
...#..
.##...
#.....
#.#..#

After 5 steps:
##.###
.##..#
.##...
.##...
#.#...
##...#
```

After 5 steps, this example now has 17 lights on.

In your grid of 100x100 lights, given your initial configuration, but with the four corners always in the on state, how many lights are on after 100 steps?

### Solution

In [5]:
with open("day18.txt", "r") as ifh:
    lights = Lights(ifh.read(), broken=True)

for idx in range(100):
    lights.update()

print(lights.brightness)
print(lights)

1006
##............................#......
.......................#.............
........................##
#.#..........................###.....
............#.#.........#............
........................##
.#..........................##.##....
............#.#.......#..#...........
..........................
...........................###.###...
...........#...#..###...#............
..........................
............................##.##....
............#..#..#.#.##.....#...##..
..........................
.............................###.....
............#..#.......#.....##..##..
...............###........
..............................#......
..............#.....###..........#...
..........................
.....................................
......................#..........###.
.............#......#.....
...................................##
.........##......................#..#
.............#.....#.#....
...................................##
........#.##..................